# Task 3 – Logistic Regression Modeling (Samad)
This notebook continues from Task 1 preprocessing.

- Model 1: Logistic Regression (full features) with GridSearchCV
- Model 2: Logistic Regression (reduced via RFE)
- Reports: params, accuracy, ROC curves, top features, overfitting check

In [ ]:

# Reuse from Task 1: preprocess, X_train, X_test, y_train, y_test
import numpy as np, pandas as pd, matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_curve, auc, classification_report, confusion_matrix
from sklearn.feature_selection import RFE

RANDOM_STATE = 42


## Model 1: Logistic Regression (Full Features)

In [ ]:

pipe_full = Pipeline([
    ("prep", preprocess),
    ("clf", LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
])

param_grid_full = {
    "clf__penalty": ["l1", "l2"],
    "clf__C": [0.01, 0.1, 1, 10],
    "clf__solver": ["liblinear", "saga"],
}

grid_full = GridSearchCV(pipe_full, param_grid_full, cv=5, scoring="accuracy", n_jobs=-1)
grid_full.fit(X_train, y_train)

ytr_pred_full = grid_full.predict(X_train)
yte_pred_full = grid_full.predict(X_test)

print("Best params:", grid_full.best_params_)
print("Train acc:", accuracy_score(y_train, ytr_pred_full))
print("Test  acc:", accuracy_score(y_test, yte_pred_full))
print("\nConfusion matrix:\n", confusion_matrix(y_test, yte_pred_full))
print("\nReport:\n", classification_report(y_test, yte_pred_full))

# Top 5 features
best_full = grid_full.best_estimator_
feat_names = best_full.named_steps["prep"].get_feature_names_out()
coefs = best_full.named_steps["clf"].coef_[0]
top5 = pd.DataFrame({"Feature": feat_names, "Coef": coefs, "Abs": np.abs(coefs)}).sort_values("Abs", ascending=False).head(5)
display(top5)

# ROC
yprob_full = best_full.predict_proba(X_test)[:,1]
fpr_full, tpr_full, _ = roc_curve(y_test, yprob_full)
auc_full = auc(fpr_full, tpr_full)
plt.plot(fpr_full, tpr_full, label=f"Full (AUC={auc_full:.3f})")


## Model 2: Logistic Regression (Reduced via RFE)

In [ ]:

base_lr = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE, solver="liblinear", penalty="l2")

pipe_rfe = Pipeline([
    ("prep", preprocess),
    ("rfe", RFE(base_lr, n_features_to_select=50, step=0.1)),
    ("clf", LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
])

param_grid_rfe = {
    "rfe__n_features_to_select": [30, 50, 80],
    "clf__penalty": ["l1", "l2"],
    "clf__C": [0.01, 0.1, 1, 10],
    "clf__solver": ["liblinear", "saga"],
}

grid_rfe = GridSearchCV(pipe_rfe, param_grid_rfe, cv=5, scoring="accuracy", n_jobs=-1)
grid_rfe.fit(X_train, y_train)

ytr_pred_rfe = grid_rfe.predict(X_train)
yte_pred_rfe = grid_rfe.predict(X_test)

print("Best params (RFE):", grid_rfe.best_params_)
print("Train acc:", accuracy_score(y_train, ytr_pred_rfe))
print("Test  acc:", accuracy_score(y_test, yte_pred_rfe))

best_rfe = grid_rfe.best_estimator_
yprob_rfe = best_rfe.predict_proba(X_test)[:,1]
fpr_rfe, tpr_rfe, _ = roc_curve(y_test, yprob_rfe)
auc_rfe = auc(fpr_rfe, tpr_rfe)
plt.plot(fpr_rfe, tpr_rfe, label=f"RFE (AUC={auc_rfe:.3f})")

plt.plot([0,1],[0,1],"--")
plt.legend(); plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC Curves Logistic Regression"); plt.show()

gap_full = accuracy_score(y_train, ytr_pred_full) - accuracy_score(y_test, yte_pred_full)
gap_rfe  = accuracy_score(y_train, ytr_pred_rfe) - accuracy_score(y_test, yte_pred_rfe)
print("Overfitting gap (Full):", gap_full)
print("Overfitting gap (RFE):", gap_rfe)


## Interpretation

In [ ]:

use_rfe = accuracy_score(y_test, yte_pred_rfe) >= accuracy_score(y_test, yte_pred_full)
chosen = best_rfe if use_rfe else best_full
model_name = "RFE" if use_rfe else "Full"

feat_names = chosen.named_steps["prep"].get_feature_names_out()
coefs = chosen.named_steps["clf"].coef_[0]
top10 = pd.DataFrame({"Feature": feat_names, "Coef": coefs, "Abs": np.abs(coefs)}).sort_values("Abs", ascending=False).head(10)
print("Top contributors from", model_name)
display(top10)
print("Positive coef -> higher odds of IsBadBuy=1 (likely kick); Negative -> lower odds.")
